In [1]:
### this program reads the consolidated trade file, makes a list of current holdings and gives delivery data for them for 5 days


In [2]:
master_file_path = "C:\\Users\\User\\Dropbox\\SQL Stock Market Data\\Python Code from ChatGPT\\Master File\\CompanyMaster.csv"
directory = "C:\\Users\\User\\Dropbox\\SQL Stock Market Data\\Data Files\\"  # Replace with the path to your directory
file_path = "C:\\Users\\User\\Dropbox\\TS Personal\\Trade Files\\Consolidated Trade File from 1st April 2022.xlsx"

output_path = "C:\\Users\\User\\Dropbox\\SQL Stock Market Data\\Python Code from ChatGPT\\Output File\\"

column_order = ['NSE_Code','BSE_Code','Date','NSE_Delivery','BSE_Delivery']


In [3]:
import os
import csv
import pandas as pd



def search_nse_code(directory, nse_code, no_days):
# Create an empty DataFrame to store the results
    nse_df = pd.DataFrame(columns=['NSE_Code', 'Date','NSE_Delivery'])
    dely_segment = ['EQ','BE','SM','ST']

    file_names = sorted([file for file in os.listdir(directory) if file.endswith('.csv') and file.startswith('NSE Delivery')], reverse=True)[:no_days]


    for file_name in file_names:
        file_path = os.path.join(directory, file_name)
        date = file_name[13:19]
            
        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            found_match = False  # Flag variable to track the match
            for i, line in enumerate(reader, start=1):
                if i >= 5:  # Start reading from line 5
                    line_str = ','.join(line)
                    data = line_str.split(',')
                    if data[2] == nse_code and data[3] in dely_segment:
                        dely = data[5]
                        nse_df = nse_df.append({'NSE_Code': nse_code, 'Date': date, 'NSE_Delivery':dely}, ignore_index=True)
                        found_match = True
                        break
            if found_match:
                continue
            

    return nse_df

def search_bse_code(directory, bse_code, no_days):
    # Create an empty DataFrame to store the results
    bse_df = pd.DataFrame(columns=['BSE_Code', 'Date','BSE_Delivery'])
    dely_segment = ['EQ','BE','SM','ST']

    file_names = sorted([file for file in os.listdir(directory) if file.endswith('.txt') and file.startswith('BSE Gross Deliverable ')], reverse=True)[:no_days]


    for file_name in file_names:
        file_path = os.path.join(directory, file_name)
        date = file_name[22:28] 
    
        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            found_match = False  # Flag variable to track the match
            for i, line in enumerate(reader, start=1):
                if i >= 2:  # Start reading from line 5
                    line_str = ','.join(line)
                    data = line_str.split('|')
                    if data[1] == bse_code:
                        dely = data[2]
                        bse_df = bse_df.append({'BSE_Code': bse_code, 'Date': date, 'BSE_Delivery':dely}, ignore_index=True)
                        found_match = True
                        break
            if found_match:
                continue
            
    return bse_df
                

def find_corresponding_codes(master_file_path, user_input):
    nse_code = ""
    bse_code = ""

    user_input = user_input.lower()  # Convert user input to lowercase for comparison

    with open(master_file_path, 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            if row[1].lower() == user_input:  # NSE code match (case-insensitive)
                nse_code = row[1]
                bse_code = str(row[2])
                break
            elif row[2].lower() == user_input:  # BSE code match (case-insensitive)
                nse_code = row[1]
                bse_code = str(row[2])
                break

    return nse_code, bse_code

def format_value(value):
    return '{:,.0f}'.format(value)

def format_decimal(x):
    return '{:,.2f}'.format(x)




In [4]:
def search_nse_price(directory, nse_code, no_days):
# Create an empty DataFrame to store the results
    nse_df = pd.DataFrame(columns=['NSE_Code', 'Date','Price','Traded_Qty'])
    dely_segment = ['EQ','BE','SM','ST']

    file_names = sorted([file for file in os.listdir(directory) if file.lower().endswith(('CSV','.csv')) and file.startswith('NSE EQ')], reverse=True)[:no_days]


    for file_name in file_names:
        file_path = os.path.join(directory, file_name)
        date = file_name[7:13]
            
        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            found_match = False  # Flag variable to track the match
            for row in reader:
                data_list = []
                data_list.append(row)
                    
                if data_list[0][0] == nse_code and data_list[0][1] in dely_segment:
                        price = data_list[0][5]
                        traded_qty = data_list[0][8]
                        nse_df = nse_df.append({'NSE_Code': nse_code, 'Date': date, 'Price':price, 
                                                'Traded_Qty': traded_qty}, ignore_index=True)
                        found_match = True
                        break
            if found_match:
                continue
            

    return nse_df

def search_bse_price(directory, bse_code, no_days):
# Create an empty DataFrame to store the results
    bse_df = pd.DataFrame(columns=['BSE_Code', 'Date','Price'])
    dely_segment = ['EQ','BE','SM','ST']

    file_names = sorted([file for file in os.listdir(directory) if file.lower().endswith(('CSV','csv')) and file.startswith('EQ ISIN')], reverse=True)[:no_days]


    for file_name in file_names:
        file_path = os.path.join(directory, file_name)
        date = file_name[8:14]
            
        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            found_match = False  # Flag variable to track the match
            for row in reader:
                data_list = []
                data_list.append(row)
                    
                if data_list[0][0] == bse_code :
                        price = data_list[0][7]
                        bse_df = bse_df.append({'BSE_Code': bse_code, 'Date': date, 'Price':price}, ignore_index=True)
                        found_match = True
                        break
            if found_match:
                continue
            

    return bse_df



In [5]:
import openpyxl

def read_sheet_pivots(file_path, sheet_name):
    data = []
    workbook = openpyxl.load_workbook(file_path, read_only=True)
    sheet = workbook[sheet_name]
    
    row_number = 28
    column_m = 'M'
    column_n = 'N'
    
    
    while True:
        cell_m = sheet[column_m + str(row_number)].value
        cell_n = sheet[column_n + str(row_number)].value
        
        if cell_m is None and cell_n is None:
            break
        
        # Convert the cell values to strings and store them in the list
        if cell_m is not None:
            m_value = str(cell_m)
            if m_value != '0' and (not m_value.isdigit() or m_value.startswith('5')):
                data.append(m_value)
        
        if cell_n is not None:
            n_value = str(cell_n)
            if n_value != '0' and (not n_value.isdigit() or n_value.startswith('5')):
                data.append(n_value)
        
        row_number += 1
    
    return data

# Example usage

sheet_name = 'Pivots'
result_list = read_sheet_pivots(file_path, sheet_name)
print(result_list)


['UJJIVANSFB', 'REMSONSIND', 'MARINE', 'SATIN', 'TRACXN', 'BLKASHYAP', 'ICICIBANK', 'KSOLVES', 'DBREALTY', 'ISGEC', 'WOCKPHARMA', 'L&tfh', 'INDIGOPNTS', 'SKIPPER', 'MAGSON', 'KTKBANK', 'KOTAKBANK', 'ARVINDFASN', 'INTENTECH', 'JPASSOCIAT', 'CESC', 'PRIMESECU', '543597', 'EDELWEISS', 'IBREALEST', 'IFCI', 'GUFICBIO', 'INDOTECH', 'TV18BRDCST', 'ADANIPOWER', 'SHRIRAMPPS', 'KFINTECH', 'TARACHAND', 'MEDPLUS', 'GOLDIAM', 'HBLPOWER', 'EIHOTEL', '542753', 'GOCOLORS', '531968', 'MANPASAND']


In [6]:
###from chatgpt -- to edit

import openpyxl
import pandas as pd

def read_sheet_pivots(file_path, sheet_name):
    data = []
    workbook = openpyxl.load_workbook(file_path, read_only=True)
    sheet = workbook[sheet_name]
    
    row_number = 28
    column_m = 'M'
    column_n = 'N'
    column_o = 'O'
    
    while True:
        cell_m = sheet[column_m + str(row_number)].value
        cell_n = sheet[column_n + str(row_number)].value
        cell_o = sheet[column_o + str(row_number)].value
        
        if cell_m is None and cell_n is None and cell_o is None:
            break
        
        if cell_m is not None:
            m_value = str(cell_m)
            if m_value != '0' and (not m_value.isdigit() or m_value.startswith('5')):
                data.append({'symbol': m_value, 'quantity': cell_o})
        
        if cell_n is not None:
            n_value = str(cell_n)
            if n_value != '0' and (not n_value.isdigit() or n_value.startswith('5')):
                data.append({'symbol': n_value, 'quantity': cell_o})
        
        row_number += 1
    
    holdings_df = pd.DataFrame(data)
    return holdings_df

# Example usage

#file_path = 'your_file_path.xlsx'  # Replace with the actual file path
sheet_name = 'Pivots'
holdings_df = read_sheet_pivots(file_path, sheet_name)
print(holdings_df)


        symbol  quantity
0   UJJIVANSFB    150000
1   REMSONSIND     12000
2       MARINE     60000
3        SATIN     15000
4       TRACXN     35000
5    BLKASHYAP     50000
6    ICICIBANK      3000
7      KSOLVES      2000
8     DBREALTY     10000
9        ISGEC      2000
10  WOCKPHARMA      5000
11       L&tfh     10000
12  INDIGOPNTS      1000
13     SKIPPER      5000
14      MAGSON     10000
15     KTKBANK      5000
16   KOTAKBANK       600
17  ARVINDFASN      2500
18   INTENTECH     10000
19  JPASSOCIAT     50000
20        CESC      7500
21   PRIMESECU      5000
22      543597      3000
23   EDELWEISS     10000
24   IBREALEST      8793
25        IFCI     25000
26    GUFICBIO      2000
27    INDOTECH      1000
28  TV18BRDCST     10000
29  ADANIPOWER      1000
30  SHRIRAMPPS      4500
31    KFINTECH      1000
32   TARACHAND      3000
33     MEDPLUS       584
34     GOLDIAM      2500
35    HBLPOWER      1000
36     EIHOTEL      1000
37      542753     50000
38    GOCOLORS        82


In [7]:
##saving the holdings to csv for uploading to chartink and trading view

# Specify the name of the CSV file you want to create
csv_file_name = output_path + "0 My_Holdings.csv"

# Write the data to the CSV file
with open(csv_file_name, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Company"])  # Optional: Write a header row
    writer.writerows([[item] for item in result_list])


In [8]:
### displaying information for multiple scrips

scrips = result_list
no_days = 7

formatted_dfs = []


for i in scrips:
    nse_code, bse_code = find_corresponding_codes(master_file_path, i)
    
    nse_result = search_nse_code(directory, nse_code,no_days)
    bse_result = search_bse_code(directory, bse_code,no_days)
    
    #### search for price info in nse/bse files 

    nse_price = search_nse_price(directory,nse_code,no_days)
    nse_price['Price'] = nse_price['Price'].astype(float)
    
    if nse_result.empty:
        if nse_code == "":
            print('stock not in nse')
        else:
            print('stock in be or st, merging price_info')
            nse_merge = pd.merge(nse_price, nse_result, on='Date', how = 'left')
            data = {'NSE_Code': nse_merge['NSE_Code_x'],
                    'Date': nse_merge['Date'],
                    'NSE_Delivery': nse_merge['Traded_Qty']
                    }
            nse_result = pd.DataFrame(data)
            
            
    if nse_result.empty:
        merged_df = pd.merge(bse_result, nse_result, on='Date', how='left')
        merged_df['NSE_Delivery'] = merged_df['NSE_Delivery'].fillna(0)
        merged_df['NSE_Code'] = merged_df['NSE_Code'].fillna("")
# Check if bse_result is empty
    elif bse_result.empty:
        merged_df = pd.merge(nse_result, bse_result, on='Date', how='left')
        merged_df['BSE_Delivery'] = merged_df['BSE_Delivery'].fillna(0)
        merged_df['BSE_Code'] = merged_df['BSE_Code'].fillna("")
# Neither nse_result nor bse_result is empty
    else:
        merged_df = pd.merge(nse_result, bse_result, on='Date', how='inner')
        
    merged_df = merged_df[column_order]

    merged_df['NSE_Delivery'] = merged_df['NSE_Delivery'].astype(int)
    merged_df['BSE_Delivery'] = merged_df['BSE_Delivery'].astype(int)
    
    merged_df['Total_Delivery'] = merged_df['NSE_Delivery'] + merged_df['BSE_Delivery']
    
    nse_price = search_nse_price(directory,nse_code,no_days)
    nse_price['Price'] = nse_price['Price'].astype(float)

    if nse_price.empty:
        bse_price = search_bse_price(directory,bse_code,no_days)
        bse_price['Price'] = bse_price['Price'].astype(float)
        final_df = pd.merge(merged_df, bse_price, on=['Date','BSE_Code'], how='inner')
    else:
        final_df = pd.merge(merged_df, nse_price, on=['Date','NSE_Code'], how = 'inner')
        final_df.drop(columns='Traded_Qty', inplace=True)
    
    
##format the df for final printing/display
    formatted_df = final_df.style.format({'NSE_Delivery': format_value, 'BSE_Delivery': format_value, 
                                          'Total_Delivery': format_value, 'Price' : format_decimal}) 


    #formatted_df
    # Append the formatted DataFrame to the list
    #formatted_dfs.append(formatted_df)
    
    # Print the formatted DataFrame
    #print(f"Formatted DataFrame for {i}:")
    #print(formatted_df)
    #print("\n")  # Add a new line for separation
    
    print(f"Formatted DataFrame for {i}:")
    display(formatted_df)
    print("\n")  # Add a new line for separation

    




Formatted DataFrame for UJJIVANSFB:


,NSE_Code,BSE_Code,Date,NSE_Delivery,BSE_Delivery,Total_Delivery,Price
0,UJJIVANSFB,542904,231206,"3,693,166","750,942","4,444,108",57.10
1,UJJIVANSFB,542904,231205,"3,619,915","176,150","3,796,065",56.60
2,UJJIVANSFB,542904,231204,"3,242,684","856,878","4,099,562",56.95
3,UJJIVANSFB,542904,231201,"2,523,839","202,956","2,726,795",56.45
4,UJJIVANSFB,542904,231130,"7,851,637","330,414","8,182,051",56.85
5,UJJIVANSFB,542904,231129,"2,667,212","437,442","3,104,654",54.45
6,UJJIVANSFB,542904,231128,"2,985,250","557,915","3,543,165",54.00




stock in be or st, merging price_info
Formatted DataFrame for REMSONSIND:


,NSE_Code,BSE_Code,Date,NSE_Delivery,BSE_Delivery,Total_Delivery,Price
0,REMSONSIND,530919,231206,"3,333","2,136","5,469",696.05
1,REMSONSIND,530919,231205,"1,282",782,"2,064",682.45
2,REMSONSIND,530919,231204,"2,680",25,"2,705",669.10
3,REMSONSIND,530919,231201,"15,746","4,600","20,346",656.00
4,REMSONSIND,530919,231130,"28,984","1,559","30,543",643.15
5,REMSONSIND,530919,231129,"1,549",25,"1,574",630.55
6,REMSONSIND,530919,231128,"1,140",37,"1,177",618.20




stock in be or st, merging price_info
Formatted DataFrame for MARINE:


...

,NSE_Code,BSE_Code,Date,NSE_Delivery,BSE_Delivery,Total_Delivery,Price
0,GOCOLORS,543401,231206,"68,239",148,"68,387","1,319.70"
1,GOCOLORS,543401,231205,"21,408",938,"22,346","1,329.15"
2,GOCOLORS,543401,231204,"37,971","1,524","39,495","1,350.35"
3,GOCOLORS,543401,231201,"16,615",532,"17,147","1,330.40"
4,GOCOLORS,543401,231130,"185,981",745,"186,726","1,329.55"
5,GOCOLORS,543401,231129,"23,396",628,"24,024","1,303.15"
6,GOCOLORS,543401,231128,"74,865","1,018","75,883","1,298.90"




stock not in nse
Formatted DataFrame for 531968:


,NSE_Code,BSE_Code,Date,NSE_Delivery,BSE_Delivery,Total_Delivery,Price
0,,531968,231206,0,200,200,17.24
1,,531968,231205,0,100,100,17.25
2,,531968,231204,0,1,1,16.99
3,,531968,231201,0,1,1,16.63
4,,531968,231130,0,112,112,17.50




stock not in nse
Formatted DataFrame for MANPASAND:


,NSE_Code,NSE_Delivery,BSE_Delivery,Total_Delivery,BSE_Code,Date,Price


In [9]:
## new code from chatgpt to check and edit


def print_quantity_symbols(symbols):
    print(", ".join(symbols))

scrips = result_list
no_days = 7

formatted_dfs = []

for i in scrips:
    nse_code, bse_code = find_corresponding_codes(master_file_path, i)

    # ... (existing code remains unchanged) ...

    # Format the df for final printing/display
    formatted_df = final_df.style.format({'NSE_Delivery': format_value, 'BSE_Delivery': format_value, 
                                          'Total_Delivery': format_value, 'Price': format_decimal})

    # Append the formatted DataFrame to the list
    formatted_dfs.append(formatted_df)

    # Print the quantity for the current symbol on top line
    quantity_symbols = [f"{i} ({formatted_df['Total_Delivery'].sum()})"]
    print_quantity_symbols(quantity_symbols)

# Print the DataFrames one by one
for i, formatted_df in zip(scrips, formatted_dfs):
    print(f"Formatted DataFrame for {i}:")
    print(formatted_df)
    print("\n")  # Add a new line for separation


TypeError: 'Styler' object is not subscriptable